In [1]:
from platform import python_version
python_version()

'3.7.12'

# Package Needed

In [1]:
import pandas as pd
import configparser
from IPython.display import JSON
from googleapiclient.discovery import build

# Preparation

In [4]:
config = configparser.ConfigParser()
config.read('../../../config.ini')

['../../../config.ini']

In [5]:
# create service parameter
api_key = config['google']['api_key']
api_service_name = 'youtube'
api_version = 'v3'

# Indonesia Belajar's Channel ID
channel_id = 'UCQ4Jo2IJeyRGzZBvjaaLzrw'

In [6]:
# create api service
youtube = build(
    serviceName=api_service_name,
    version=api_version,
    developerKey=api_key)

# Request Channel

In [14]:
request = youtube.channels().list(
          part="snippet,contentDetails,statistics",
          id=channel_id)
response = request.execute()

In [15]:
JSON(response)

<IPython.core.display.JSON object>

# Request Video Id using Playlist

In [28]:
playlist_id = "UUQ4Jo2IJeyRGzZBvjaaLzrw"
video_ids = []
request = youtube.playlistItems().list(
            part = "contentDetails,snippet",
            playlistId = playlist_id,
            maxResults = 50)
response = request.execute()

for video_id in response["items"]:
    video_ids.append(video_id["contentDetails"]["videoId"])

next_page_token = response.get("nextPageToken")

while next_page_token is not None:
    request = youtube.playlistItems().list(
                part = "contentDetails,snippet",
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = next_page_token)
    response = request.execute()
    for video_id in response["items"]:
        video_ids.append(video_id["contentDetails"]["videoId"])
        
    next_page_token = response.get("nextPageToken")

In [31]:
len(video_ids)

501

# Request Every Video Detail

In [44]:
request = youtube.videos().list(
            part="contentDetails,snippet,statistics",
            id = video_ids[0])
response = request.execute()
JSON(response)

<IPython.core.display.JSON object>

In [56]:
video_detail = []
for item in response["items"]:
    vid_id = item["id"]
    title = item["snippet"]["title"]
    description = item["snippet"]["description"]
    date_published = item["snippet"]["publishedAt"]
    tags = [tag for tag in item["snippet"]["tags"]]
    duration = item["contentDetails"]["duration"]
    view = item["statistics"]["viewCount"]
    like = item["statistics"]["likeCount"]
    video_detail.append({"id":vid_id, "title":title, "description":description, "date_published":date_published, 
                         "tags":tags, "duration":duration, "view":view, "like":like})

In [58]:
len(video_detail)

1

In [62]:
video_details = []
for vid_numb in range(0, len(video_ids), 50):
    request = youtube.videos().list(
                part="contentDetails,snippet,statistics",
                id = ",".join(video_ids[vid_numb:vid_numb+50]))
    response = request.execute()
    
    for item in response["items"]:
        vid_id = item["id"]
        title = item["snippet"]["title"]
        description = item["snippet"]["description"]
        date_published = item["snippet"]["publishedAt"]
        try:
            tags = [tag for tag in item["snippet"]["tags"]]
        except:
            tags = None
        duration = item["contentDetails"]["duration"]
        view = item["statistics"]["viewCount"]
        like = item["statistics"]["likeCount"]
    
        video_details.append({"id":vid_id, "title":title, "description":description, "date_published":date_published, 
                             "tags":tags, "duration":duration, "view":view, "like":like})

In [64]:
df = pd.DataFrame(video_details)
df.head()

,id,title,description,date_published,tags,duration,view,like
0,qgD4hlioVWA,Rebase Fedora Silverblue | Fedora Kinoite | So...,Salam Indonesia Belajar!!!\n\nRebase Immutable...,2022-05-12T09:00:53Z,"[fedora kinoite, fedora silverblue, fedora lin...",PT29M13S,317,20
1,LW1YtdZKeDA,CSD 16 | Tips Belajar Programming dan Belajar ...,"Hallo sobat Indonesia Belajar, jumpa lagi deng...",2022-05-07T02:00:12Z,"[catatan seorang dosen, indonesia belajar, tip...",PT16M52S,1779,104
2,G8ULufXAUvE,TS 08 | Mengenal Decomposition | De-Seasonalis...,Salam Indonesia Belajar!!! \n\nMengenal Decomp...,2022-04-25T09:00:11Z,"[time series, data deret waktu, belajar time s...",PT31M35S,1202,17
3,IhW5vTjo9LE,CSD 15 | Sidang Skripsi Online di masa Pandemi...,"Hallo sobat Indonesia Belajar, jumpa lagi deng...",2022-04-21T09:00:13Z,"[sidang skripsi, sidang online, sidang skripsi...",PT9M12S,615,34
4,kb1EKafVl1w,Seputar Blockchain bersama Nur Arifin Akbar | ...,Bincang santai seputar Blockchain bersama Nur ...,2022-04-18T09:00:22Z,"[nur arifin akbar, nur arifin blockchain, bank...",PT42M33S,682,33


In [66]:
df.to_csv("all video from Indonesia Belajar.csv", index=False, sep=";")

In [14]:
# save json file
# import json

# with open('list semua playlist.json', 'w') as playlist_json:
#     json.dump(response, playlist_json)

### JSON Playlist Data Pre-Processing

In [15]:
open_json_file = open('list semua playlist.json')
playlist_items = json.load(open_json_file)

In [16]:
JSON(playlist_items)

<IPython.core.display.JSON object>

In [17]:
playlist_items_id = playlist_items['items'][0]['id']
playlist_items_title = playlist_items['items'][0]['snippet']['title']
playlist_items_description = playlist_items['items'][0]['snippet']['description']
playlist_video_total = playlist_items['items'][0]['contentDetails']

In [18]:
playlist_items_id

'PL2O3HdJI4voFWT9zTM2wUqHDLt2Ds_3bi'

In [19]:
playlist_id = []
playlist_title = []
playlist_description = []
playlist_video_total = []
for item in playlist_items['items']:
    playlist_id.append(item['id'])
    playlist_title.append(item['snippet']['title'])
    playlist_description.append(item['snippet']['description'])
    playlist_video_total.append(item['contentDetails'])

playlist_dict = {
    'id': playlist_id,
    'title': playlist_title,
    'description': playlist_description,
    'video_total': playlist_video_total}

In [20]:
df_playlist = pd.DataFrame(playlist_dict)

In [21]:
df_playlist.head()

,id,title,description,video_total
0,PL2O3HdJI4voFWT9zTM2wUqHDLt2Ds_3bi,Podcast Blockchain,,{'itemCount': 4}
1,PL2O3HdJI4voED2tsGDA2MRjKIhyjhHKNK,Podcast Data,,{'itemCount': 7}
2,PL2O3HdJI4voHK5d2Vom-MKQ2IVcxcTLPQ,Belajar Container bersama AWS,,{'itemCount': 4}
3,PL2O3HdJI4voEQS7Ka4RxHRAWJz_NIymkz,Belajar Container bersama AWS,,{'itemCount': 0}
4,PL2O3HdJI4voFyH616uejhvYt5AQzZ2iTi,Belajar Time Series dengan Google Sheets,,{'itemCount': 7}


### Cleaning Data

In [23]:
df_playlist['video_total'] = df_playlist['video_total'].astype(str)
df_playlist['video_total'] = df_playlist['video_total'].str.replace('({\'itemCount\': |})', '').str.strip()

C:\Users\Rizky\AppData\Local\Temp/ipykernel_11904/2337346490.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_playlist['video_total'] = df_playlist['video_total'].str.replace('({\'itemCount\': |})', '').str.strip()


In [24]:
df_playlist.head()

,id,title,description,video_total
0,PL2O3HdJI4voFWT9zTM2wUqHDLt2Ds_3bi,Podcast Blockchain,,4
1,PL2O3HdJI4voED2tsGDA2MRjKIhyjhHKNK,Podcast Data,,7
2,PL2O3HdJI4voHK5d2Vom-MKQ2IVcxcTLPQ,Belajar Container bersama AWS,,4
3,PL2O3HdJI4voEQS7Ka4RxHRAWJz_NIymkz,Belajar Container bersama AWS,,0
4,PL2O3HdJI4voFyH616uejhvYt5AQzZ2iTi,Belajar Time Series dengan Google Sheets,,7


In [25]:
df_playlist.to_csv('list semua playlist.csv', index=False)